In [4]:
import pandas as pd

In [5]:
df = pd.read_pickle('domain_adaptation_english/data/yelp_dataset_reviews.pkl')
df.head()

,text
0,"If you decide to eat here, just be aware it is..."
1,I've taken a lot of spin classes over the year...
2,Family diner. Had the buffet. Eclectic assortm...
3,"Wow! Yummy, different, delicious. Our favo..."
4,Cute interior and owner (?) gave us tour of up...


In [6]:
texts = df['text'].tolist()[:100]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [8]:
tokens = tokenizer.tokenize(texts[0])

pretok_sent = ""
for tok in tokens:
     if tok.startswith("##"):
         pretok_sent += tok[2:]
     else:
         pretok_sent += " " + tok

print(texts[0])
print(pretok_sent)
print('testing diffrence when tokenezing agian')
print(len(tokens))
print(len(tokenizer.tokenize(pretok_sent)))

If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. 

The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.
 if you decide to eat here , just be aware it is going to take about 2 hours from beginning to end . we have tried it multiple times , because i want to like it ! i have been to it ' s other locations in nj and never had a bad experience . the food is good , but it takes a very long time to come out . the waitstaff is very young , but usually pleasant . we have just had too many experiences where we spent way too long waiting . we usually opt for another diner or restaurant on the 

## complex

In [9]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [10]:
from math import ceil
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


In [26]:
def pretokenize(tokens):
    '''
    return tokenized text to a string
    '''
    pretok_sent = ''
    for tok in tokens:
        if tok.startswith('##'):
            pretok_sent += tok[2:]
        else:
            pretok_sent += ' ' + tok
    return pretok_sent


def token_split(tokens, max_size):
    '''
    split tokens after toknezing them
    '''
    num_of_splits = ceil(len(tokens) / max_size)
    split_tokens_text = [pretokenize(tokens[(max_size*i):(max_size*(i+1))]) for i in range(num_of_splits)]
    return split_tokens_text

def split_inputs(input_texts, max_size = 128):
    '''
    input_texts: a list of input texts.
    max_size: the max size to truncate data to.
    '''
    nlp = spacy.load("en_core_web_sm")
    new_input_texts = []
    orginal_indexs = []
    
    for i, doc in enumerate(nlp.pipe(input_texts, disable=["tagger", "attribute_ruler", "lemmatizer"])):
        sentences = [sent.text for sent in doc.sents]
        new_sentences = []
        for sent in sentences:
            new_sentences.extend([split_sent for split_sent in token_split(tokenizer.tokenize(sent), max_size)] )
        orginal_indexs.extend([i for sent in new_sentences])
        new_input_texts.extend(new_sentences)

    return new_input_texts, orginal_indexs

def join_outputs(split_outputs, orginal_indexs):
    '''
    join splited outputs
    '''
    if not orginal_indexs:
        raise Exception('split inputs indexes are empty')
    if not split_outputs:
        raise Exception('split outputs are empty')

    joined_outputs = []
    output = split_outputs[0]
    len_idxs = len(orginal_indexs)
    for i in range(len_idxs-1):
        past_index = orginal_indexs[i]
        current_index = orginal_indexs[i+1]
        if i == (len_idxs-1):
            joined_outputs.append(output)
        if current_index ==  past_index:
            output.extend(split_outputs[i+1])
        else:
            joined_outputs.append(output)
            output = split_outputs[i+1]
    joined_outputs.append(output)
    return joined_outputs

In [27]:
new_input_texts, orginal_index = split_inputs(texts)

In [28]:
orginal_index[-10:]

[99, 99, 99, 99, 99, 99, 99, 99, 99, 99]

In [29]:
new_input_texts[:10]

[' if you decide to eat here , just be aware it is going to take about 2 hours from beginning to end .',
 ' we have tried it multiple times , because i want to like it !',
 " i have been to it ' s other locations in nj and never had a bad experience .",
 ' the food is good , but it takes a very long time to come out .',
 ' the waitstaff is very young , but usually pleasant .',
 ' we have just had too many experiences where we spent way too long waiting .',
 ' we usually opt for another diner or restaurant on the weekends , in order to be done quicker .',
 " i ' ve taken a lot of spin classes over the years , and nothing compares to the classes at body cycle .",
 ' from the nice , clean space and amazing bikes , to the welcoming and motivating instructors , every class is a top notch work out .',
 " for anyone who struggles to fit workouts in , the online scheduling system makes it easy to plan ahead ( and there ' s no need to line up way in advanced like many gyms make you do ) ."]

In [30]:
new_input_texts = [[x] for x in new_input_texts]

In [31]:
joined = join_outputs(new_input_texts, orginal_index)

In [32]:
len(joined)

100

In [53]:
def t():
    x = [1, 2, 3]
    y = [3, 4, 5]
    z = [6, 7, 8]
    return x, y, z
# r = [x, y, z]
g = t()
list(g)[0]

[1, 2, 3]

In [ ]:
orginal_index[:10]

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

In [ ]:
new_input_texts[:7]

[' if you decide to eat here , just be aware it is going to take about 2 hours from beginning to end .',
 ' we have tried it multiple times , because i want to like it !',
 " i have been to it ' s other locations in nj and never had a bad experience .",
 ' the food is good , but it takes a very long time to come out .',
 ' the waitstaff is very young , but usually pleasant .',
 ' we have just had too many experiences where we spent way too long waiting .',
 ' we usually opt for another diner or restaurant on the weekends , in order to be done quicker .']

## simple

In [ ]:
def sent_split(tokens, max_size):
    num_of_splits = ceil(len(tokens) / max_size)
    split_tokens_text = [tokens[(max_size*i):(max_size*(i+1))] for i in range(num_of_splits)]
    return split_tokens_text

def normlize_bert_size(input_texts, max_size = 512):
    '''
    input_texts: a list of input texts.
    max_size: the max size to truncate data to.
    '''
    nlp = spacy.load("en_core_web_sm")
    new_input_texts = []
    orginal_index = []
    
    for i, doc in enumerate(nlp.pipe(input_texts, disable=["tagger", "attribute_ruler", "lemmatizer"])):
        sentences = [sent.text for sent in doc.sents]
        new_sentences = []
        for sent in sentences:
            new_sentences.extend([split_sent for split_sent in sent_split(tokenizer.tokenize(sent), max_size)] )
        orginal_index.extend([i for sent in new_sentences])
        new_input_texts.extend(new_sentences)

    return new_input_texts, orginal_index

In [ ]:
new_input_texts, orginal_index = normlize_bert_size(texts, max_size=400)

the simple method is around 10% faster than the complex method

In [43]:
# count = 0
# for i, text in enumerate(texts):
#     sentences = text.split('\n')
#     if len(sentences) > 1:
#         print(len(sentences))
#         print(sentences)
#         print(text)
#         count += 1
#         if count == 10:
#             break

In [49]:
count = 0
for i, doc in enumerate(nlp.pipe(texts, disable=["tagger", "attribute_ruler", "lemmatizer"])):
    sentences = [sent.text for sent in doc.sents]
    # print([sent for sent in doc.sents])
    if len(sentences) > 1:
        print(sentences)
        print(doc.text)
        count += 1
        if count == 10:
            break

['If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end.', 'We have tried it multiple times, because I want to like it!', "I have been to it's other locations in NJ and never had a bad experience.", '\n\nThe food is good, but it takes a very long time to come out.', 'The waitstaff is very young, but usually pleasant.', 'We have just had too many experiences where we spent way too long waiting.', 'We usually opt for another diner or restaurant on the weekends, in order to be done quicker.']
If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. 

The food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or resta

In [48]:
# count = 0
# for i, doc in enumerate(nlp.pipe(texts, disable=["tok2vec", "tagger", "attribute_ruler", "lemmatizer"])):
#     sentences = doc.sents
#     print([sent for sent in doc.sents])
#     if len([sent for sent in doc.sents]) > 1:
#         print([sent for sent in doc.sents])
#         print(doc.text)
#         count += 1
#         if count == 10:
#             break